<a href="https://colab.research.google.com/github/emrapport/w266-final-project/blob/master/CNN_with_equal_sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

11/3/2019

In [0]:
# this notebook is based off of this blog post: 
# https://realpython.com/python-keras-text-classification/#reader-comments

In [0]:
DATASET_NAME = "contractions"
MODEL_NAME = "contractions"
MAX_SEQ_LENGTH = 20
NUM_MALE_EXAMPLES = 2000000
NUM_FEMALE_EXAMPLES = 2000000
VAL_SET_SIZE = 1000000
NUM_EPOCHS = 15
BATCH_SIZE = 1000

# I don't think we have a good reason to do this right now
# but we might eventually 
SAVE_TOKENIZED_DATA = False

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import HashingVectorizer
from scipy.sparse import hstack, vstack
from keras.preprocessing.text import Tokenizer
from tensorflow.keras import Sequential, layers
from keras.utils import plot_model
import pandas as pd
import numpy as np
import copy
import time
import pickle
!pip install gcsfs

pd.set_option('max_colwidth', 100)

Using TensorFlow backend.


In [4]:
# this cell is only necessary if running in colab
project_id = 'w266-251323'
import uuid
bucket_name = 'fb-congressional-data/'
from google.colab import auth
auth.authenticate_user()
!gcloud config set project {project_id}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Updated property [core/project].


In [7]:
train_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/train.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [8]:
dev_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/dev.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [0]:
# shuffle the data
# be sure to do this before you extract X's and y's!!
train_df = train_df.sample(frac=1)
dev_df = dev_df.sample(frac=1)

In [12]:
### here is where we do some simple sampling
male_train_rows = train_df[train_df.op_gender=='M']
female_train_rows = train_df[train_df.op_gender=='W']

male_train_to_use = male_train_rows[:2000000]
female_train_to_use = female_train_rows[:2000000]
print(len(male_train_to_use))
print(len(female_train_to_use))

combined_train_df = pd.concat([male_train_to_use, female_train_to_use])
combined_train_df = combined_train_df.sample(frac=1)
combined_train_df.head()

2000000
2000000


,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category
5525938,7326558,60239508,W,274542,James,bobbi I am the christian who will pray for your salvation.,Amy Klobuchar,Congress_Democratic
884341,884341,29687946,M,13607,Steven,keep it simple rand!,Rand Paul,Congress_Republican
1970696,2043872,61587478,M,32091,Krikor,"enough is enough with governor brown and obama. taking our money to fund their personal desire, ...",Jeff Denham,Congress_Republican
4031349,5507785,42443414,W,175145,Patty,you asked great questions. she gave poor responses.,Lynn Jenkins,Congress_Republican
1250486,1250486,29687946,M,14081,Stacy,my son just did a report on ph friday.,Rand Paul,Congress_Republican


In [0]:
y_train = combined_train_df.op_gender.values
y_dev = dev_df.op_gender.values

In [0]:
def turn_to_ints(li):
    final_list = []
    for gender in li:
        if gender=='M':
            final_list.append(1)
        else:
            final_list.append(0)
    return final_list
            
y_train = turn_to_ints(y_train)
y_dev = turn_to_ints(y_dev)

In [0]:
y_train = np.asarray(y_train)
y_dev = np.asarray(y_dev)

In [0]:
def get_text_list(init_list):
    sentences = []
    for sentence in init_list:
        if type(sentence) != str:
            sentences.append("")
        else:
            sentences.append(sentence)
    return sentences

new_sentences_train = get_text_list(combined_train_df.response_text.values)
new_sentences_test = get_text_list(dev_df.response_text.values)

In [17]:
time_start = time.time()

# this is the default list of filters + apostrophe
# added because we have dealt with common contractions, so other apostrophes should mostly be possessive 
tokenizer = Tokenizer(num_words=200000, filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(new_sentences_train)

X_train = tokenizer.texts_to_sequences(new_sentences_train)
X_test = tokenizer.texts_to_sequences(new_sentences_test)

vocab_size = len(tokenizer.word_index) + 1

currentTime = time.gmtime(time.time() - time_start)

#Convert the gmtime struct to a string
timeStr = time.strftime("%M minutes, %S seconds", currentTime)

print("Tokenized in {}".format(timeStr))

Tokenized in 04 minutes, 57 seconds


In [0]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 20

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [0]:
if SAVE_TOKENIZED_DATA:
  x_train_path = 'X_train_{}.pkl'.format(DATASET_NAME)
  x_dev_path = 'X_dev_{}.pkl'.format(DATASET_NAME)
  y_train_path = 'y_train_{}.pkl'.format(DATASET_NAME)
  y_dev_path = 'y_dev_{}.pkl'.format(DATASET_NAME)

  with open(x_train_path, 'wb') as file:
      pickle.dump(X_train, file)   
  with open(x_dev_path, 'wb') as file:
      pickle.dump(X_test, file)
  with open(y_train_path, 'wb') as file:
      pickle.dump(y_train, file)
  with open(y_dev_path, 'wb') as file:
      pickle.dump(y_dev, file)

  # copy to bucket
  !gsutil cp /content/{x_train_path} gs://fb-congressional-data/test
  !gsutil cp /content/{x_dev_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_train_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_dev_path} gs://fb-congressional-data/

In [19]:
!gsutil cp gs://fb-congressional-data/glove* /tmp/

Copying gs://fb-congressional-data/glove.6B.100d.txt...
Copying gs://fb-congressional-data/glove.6B.200d.txt...
Copying gs://fb-congressional-data/glove.6B.300d.txt...
Copying gs://fb-congressional-data/glove.6B.50d.txt...
\ [4 files][  2.1 GiB/  2.1 GiB]   22.6 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://fb-congressional-data/glove.6B.zip...
- [5 files][  2.9 GiB/  2.9 GiB]   24.6 MiB/s                                   
Operation completed over 5 objects/2.9 GiB.                                      


In [0]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

embedding_dim = 100
embedding_matrix = create_embedding_matrix(
                     '/tmp/glove.6B.{}d.txt'.format(embedding_dim),
                      tokenizer.word_index, embedding_dim)

In [21]:
# trying to figure out which words are empty here
counter = 0
empty_indexes = []
for index, row in enumerate(embedding_matrix):
  if sum(row) == 0:
    empty_indexes.append(index)
    counter += 1
  if counter > 1000:
    break

for idx in empty_indexes:
  try:
    print(tokenizer.index_word[idx])
  except:
    print("No entry for {}".format(idx))


No entry for 0
stillsanders
youtu
fbid
•
palestinei
🇺🇸
standwithrand
👍
🇺🇸🇺🇸
obummer
feelthebern
huffingtonpost
rinos
❤️
lmao
washingtonpost
killary
doyourjob
fbcdn
bernieorbust
neverhillary
😍
hphotos
hahaha
😊
isil
nobillnobreak
sheeple
👎
berniesanders
bridenstine
repub
cispa
thehill
1073741828
bengazi
randrally
❤
trump2016
😡
😂
libtards
aspx
drumpf
dailycaller
scontent
randpaul
obozo
███
thinkprogress
§
fref
sphotos
bernie2016
agsaf
politicususa
bonamici
akamaihd
╬═╬
banksters
hilliary
westernjournalism
libtard
washingtontimes
teabaggers
nationalreview
╚═
═╝
stopp
hahahaha
kobanê
2366
demandavote
♥
opensecrets
teaparty
2fwww
🇺🇸🇺🇸🇺🇸
koolaid
imwithher
notif
nodapl
someunknownushistory
dapl
nobama
imho
♡
😉
shouldnt
😀
nevertrump
͡°
infowars
oboma
washingtonexaminer
cnsnews
occupydemocrats
على
conservativetribune
devoss
daca
0bama
wtg
democrates
repugs
benifits
wenstrup
gofundme
dumbass
ovomit
govtrack
worstresponders
pnref
p71bjjcp8
abcnews
democraps
lieing
🙏
ijreview
shillary
kxl
djt
kille

In [22]:
# hmmmm....
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
nonzero_elements / vocab_size

0.2328965752257625

In [23]:
embedding_matrix.shape

(586900, 100)

In [24]:
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=maxlen, 
                           trainable=False))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 100)           58690000  
_________________________________________________________________
conv1d (Conv1D)              (None, 16, 128)           64128     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                1290      
_________________________________________________________________
dense_1 (Dense)            

In [0]:
smaller_X_dev = X_test[:VAL_SET_SIZE]
smaller_y_dev = y_dev[:VAL_SET_SIZE]

In [27]:
try:
  time_start = time.time()

  history = model.fit(X_train, y_train,
                      epochs=NUM_EPOCHS,
                      verbose=True,
                      validation_data=(smaller_X_dev, smaller_y_dev),
                      batch_size=BATCH_SIZE)

  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))

except:
  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))  

Train on 4000000 samples, validate on 1000000 samples
Epoch 1/15
4000000/4000000 [==============================] - 104s 26us/sample - loss: 0.5971 - acc: 0.6461 - val_loss: 0.6185 - val_acc: 0.6592
Epoch 2/15
4000000/4000000 [==============================] - 106s 26us/sample - loss: 0.5791 - acc: 0.6623 - val_loss: 0.6785 - val_acc: 0.5841
Epoch 3/15
4000000/4000000 [==============================] - 106s 27us/sample - loss: 0.5732 - acc: 0.6676 - val_loss: 0.6563 - val_acc: 0.6033
Epoch 4/15
4000000/4000000 [==============================] - 108s 27us/sample - loss: 0.5694 - acc: 0.6708 - val_loss: 0.6416 - val_acc: 0.6370
Epoch 5/15
4000000/4000000 [==============================] - 110s 27us/sample - loss: 0.5667 - acc: 0.6733 - val_loss: 0.6355 - val_acc: 0.6373
Epoch 6/15
4000000/4000000 [==============================] - 108s 27us/sample - loss: 0.5645 - acc: 0.6752 - val_loss: 0.6739 - val_acc: 0.5878
Epoch 7/15
4000000/4000000 [==============================] - 108s 27us/samp

In [29]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_dev, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


Training Accuracy: 0.6873
Testing Accuracy:  0.5987


In [0]:
# can't get this to work
plot_model(history)

In [0]:
preds = model.predict(X_test)

In [0]:
dev_df['probs'] = preds

In [0]:
def pred_to_label(row):
  if row['probs'] >= .5:
    return 'M'
  else:
    return 'W'

dev_df['preds'] = dev_df.apply(pred_to_label, axis=1)

In [0]:
wrong_preds = dev_df[dev_df.op_gender!=dev_df.preds]

In [35]:
wrong_preds.op_gender.value_counts()

M    764759
W    155383
Name: op_gender, dtype: int64

In [34]:
# total proportion of preds for women
dev_df.preds.value_counts()['W'] / len(dev_df)

0.41602733996625246

In [36]:
#random sample of mistakes
wrong_preds.head(100)

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
1646369,10127228,51495713,M,370484,Ashley,"good luck on your upcoming election, hope everything turns out good . you have always done what ...",Richard Shelby,Congress_Republican,0.387662,W
1762189,10285326,32570461,M,382123,Cheri,"our country was based on christian morals. for our grandchildrens' sake, please do not think th...",Tim Kaine,Congress_Democratic,0.369875,W
2281596,13465058,36181883,M,517476,Karen,beautiful cover,Steve Scalise,Congress_Republican,0.295095,W
565189,4814243,46506154,M,160148,Darin,at this point she needs to be striped of hers. punish her minions? big deal,Marco Rubio,Congress_Republican,0.307168,W
1517997,7936291,99469231,M,296389,Greto,lynda balzer is a fox parrot with no brains.,John McCain,Congress_Republican,0.456124,W
...,...,...,...,...,...,...,...,...,...,...
498076,4080275,77914417,M,142602,George,barukh dayyan haemet.,John Garamendi,Congress_Democratic,0.478319,W
1535719,7954013,99469231,M,296478,Steven,congratulations arizona from from your friends in the palmetto state.,John McCain,Congress_Republican,0.158420,W
1603547,10051417,49028900,M,365919,Louis,my fellow republicans better realize that they vote rep in the next election the clock is going ...,Steve King,Congress_Republican,0.477389,W
1267739,7591553,54387439,M,291493,Kristen,yea !!!!!! heather pfancook,Ryan Zinke,Congress_Republican,0.375056,W


In [37]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=False)[:100]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
1993131,11042523,86569077,W,389977,Colleen,you mean mr paul mark? rand opportunist paul?,Kirsten Gillibrand,Congress_Democratic,0.999768,M
2261469,13332806,62264117,W,511603,Lisa,"he has a great officer, leader, and friend. i know he will keep moving up. congrats sir and ho...",Grace Napolitano,Congress_Democratic,0.999194,M
2030227,11079619,86569077,W,390346,John,larry north do you mean senator chuck you schumer ?,Kirsten Gillibrand,Congress_Democratic,0.999069,M
2043732,11093124,86569077,W,390564,John,"@jimmy, with all due respect, i prefer a ticket of franken-gillibrand in 2016.",Kirsten Gillibrand,Congress_Democratic,0.999059,M
2136861,11277841,77234050,W,399896,Carol,"please senator mcmorris, support rand paul in his efforts to protect our rights to due process",Cathy McMorris Rodgers,Congress_Republican,0.998916,M
...,...,...,...,...,...,...,...,...,...,...
1999882,11049274,86569077,W,390038,Miro,just the right hand gal for senator chucky cheeze schumer.,Kirsten Gillibrand,Congress_Democratic,0.994671,M
1950395,10999787,86569077,W,389553,Claire,i am angry and devastated that my brother kevin lynch was not allowed to receive coverage for t...,Kirsten Gillibrand,Congress_Democratic,0.994667,M
2290366,13626031,4938109,W,528810,William,it was always racism from day one. anything to get back at the best president america has ever h...,Yvette Clarke,Congress_Democratic,0.994664,M
1993458,11042850,86569077,W,389980,Kristin,"that is why i stand by rand. but you must be constitutional across the bridge and back, not sel...",Kirsten Gillibrand,Congress_Democratic,0.994661,M


In [38]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=True)[:100]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
1154104,5870662,12888814,M,195530,Nancy,glad you are not my congress person. congresswoman schakowsky put people over party.,Don Beyer,Congress_Democratic,0.000012,W
991634,5240688,46506154,M,161317,Laura,"keep the faith ms. ayotte, you are good for new hampshire and our country !! you have more supp...",Marco Rubio,Congress_Republican,0.000020,W
1833199,10356336,55333293,M,383488,Natalia,greetings from puerto rico! god bless ms. warren,Peter DeFazio,Congress_Democratic,0.000050,W
181521,2358273,62246450,M,52502,Annie,keep going elizabeth! we support you!,Ron Wyden,Congress_Democratic,0.000070,W
1883166,10916094,10458808,M,385912,Stephen,but takano does not support kamala harris! he just lost my vote!,Mark Takano,Congress_Democratic,0.000072,W
...,...,...,...,...,...,...,...,...,...,...
901076,5150130,46506154,M,161080,Mark-Terry,"really patty, what bolt did you not screw in all the way?",Marco Rubio,Congress_Republican,0.000835,W
207982,2384734,62246450,M,52705,Cyndy,"me too, i love elizabeth warren!",Ron Wyden,Congress_Democratic,0.000847,W
1705552,10211156,32185537,M,376624,David,way to go zoe!!! you will do great things.,Paul Gosar,Congress_Republican,0.000850,W
1878597,10911525,10458808,M,385781,Rose,love you debbie,Mark Takano,Congress_Democratic,0.000856,W


In [0]:
# preds in the middle range
borderline_wrong_preds_f = wrong_preds[wrong_preds.probs.between(.4, .5)]

In [50]:
# this is how many wrong preds were in the "borderline" range
# where model predicted female instead of male
wrong_preds.shape[0] - borderline_wrong_preds_f.shape[0]

532583

In [51]:
borderline_wrong_preds_f.head(40)

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
1517997,7936291,99469231,M,296389,Greto,lynda balzer is a fox parrot with no brains.,John McCain,Congress_Republican,0.456124,W
1089384,5338438,46506154,M,161595,Merrill,marco is an honest man among thieves.,Marco Rubio,Congress_Republican,0.461238,W
485805,4068004,77914417,M,140830,Melissa,we support you!!!,John Garamendi,Congress_Democratic,0.432142,W
1347494,7765788,99469231,M,296125,Joan,i cannot vote for this republican. it is the first time ever for me.,John McCain,Congress_Republican,0.468812,W
1569358,7987652,99469231,M,296805,Donna,lol we seeee u :),John McCain,Congress_Republican,0.478015,W
17231,2135146,38794897,M,36364,Ghii,all inner city students should be sent to boarding schools to escape their poor environment,Sherrod Brown,Congress_Democratic,0.461489,W
655905,4904959,46506154,M,160224,Ben,https://www.facebook.com/1487587521541634/photos/a.1489348611365525.1073741828.1487587521541634/...,Marco Rubio,Congress_Republican,0.467257,W
510123,4759177,46506154,M,159898,Toni,what a shame.,Marco Rubio,Congress_Republican,0.447867,W
1215197,6674381,77657070,M,255878,Joseph,"now, get the subsidies for large corporate farms killed so the small guy can compete.",Mike Crapo,Congress_Republican,0.425737,W
250913,2506112,23785853,M,62153,Howard,"besides pushing to roll back dodd-frank, trump ordered a delay of the ""fiduciary rule"" that conc...",Mike Simpson,Congress_Republican,0.471734,W


In [52]:
borderline_wrong_preds_m = wrong_preds[wrong_preds.probs.between(.5, .6)]

# this is how many wrong preds were in the "borderline" range
# where model predicted female instead of male
wrong_preds.shape[0] - borderline_wrong_preds_m.shape[0]

841380

In [53]:
borderline_wrong_preds_m.head(40)

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
2079857,11220837,77234050,W,399123,Diane,i just want to let you know that i am disabled after a work history of 35 years. i certainly fou...,Cathy McMorris Rodgers,Congress_Republican,0.519979,M
2064665,11204419,48870736,W,394934,Mark,make sure those factory farms get held accountable....,Betty McCollum,Congress_Democratic,0.540100,M
2064351,11204105,48870736,W,394926,Michael,come join us at the arc protest today in bloomington at 4pm! 😎,Betty McCollum,Congress_Democratic,0.587165,M
2103549,11244529,77234050,W,399377,Marcus,pass small targeted single issue bills and force obama to respond to our agenda.,Cathy McMorris Rodgers,Congress_Republican,0.581134,M
2076896,11217876,77234050,W,398805,Mike,data....???,Cathy McMorris Rodgers,Congress_Republican,0.557267,M
2066788,11206542,48870736,W,395083,Jim,"wow, what a bunch of ?????%$&",Betty McCollum,Congress_Democratic,0.541130,M
2002232,11051624,86569077,W,390051,John,"jennifer, if the senator has a legal firearm in her home for self-protection, why not? she does ...",Kirsten Gillibrand,Congress_Democratic,0.530886,M
1909675,10959067,86569077,W,389337,Jenny,proud to be a new yorker these days! keep it up!,Kirsten Gillibrand,Congress_Democratic,0.518922,M
2012027,11061419,86569077,W,390124,Stanley,"1. ban assault weapons, and oversize ammo clips. that has a good first step. 2. require gun lic...",Kirsten Gillibrand,Congress_Democratic,0.598196,M
1967743,11017135,86569077,W,389716,Bill,have a proper investigation and release the 28 pages that were redacted by the 911 tainted team.,Kirsten Gillibrand,Congress_Democratic,0.563130,M
